# Setup model

In [ ]:
import os
import sys
from datetime import datetime

import numpy as np
import pandas as pd

import keras

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import RandomTranslation

import matplotlib.pyplot as plt

%matplotlib inline

seed = 9
np.random.seed(seed)
tf.random.set_seed(seed)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")
BATCH_SIZE = 64

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
tf.get_logger().setLevel('ERROR')
sys.stderr = open(os.devnull, 'w')

# Visualization

In [ ]:
def create_segmentation_colormap(num_classes):
    """
    Create a linear colormap using a predefined palette.
    Uses 'viridis' as default because it is perceptually uniform
    and works well for colorblindness.
    """
    return plt.cm.viridis(np.linspace(0, 1, num_classes))

def apply_colormap(label, colormap=None):
    """
    Apply the colormap to a label.
    """
    # Ensure label is 2D
    label = np.squeeze(label)

    if colormap is None:
        num_classes = len(np.unique(label))
        colormap = create_segmentation_colormap(num_classes)

    # Apply the colormap
    colored = colormap[label.astype(int)]

    return colored

class VizCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset, frequency=5, num_classes=2):
        super().__init__()
        self.dataset = dataset
        self.frequency = frequency
        self.num_classes = num_classes
        self.dataset_iter = iter(dataset)  # Crea un iteratore per accedere ai dati

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.frequency == 0:  # Visualizza solo ogni "frequency" epochs
            try:
                # Estrai un batch di dati
                image, label = next(self.dataset_iter)
            except StopIteration:
                # Ricrea l'iteratore se i dati sono terminati
                self.dataset_iter = iter(self.dataset)
                image, label = next(self.dataset_iter)

            # Prepara i dati per la predizione
            image = tf.expand_dims(image[0], 0)  # Estrai una sola immagine dal batch
            label = label[0]  # Etichetta corrispondente
            pred = self.model.predict(image, verbose=0)
            y_pred = tf.math.argmax(pred, axis=-1)
            y_pred = y_pred.numpy()

            # Creazione della mappa colori
            colormap = create_segmentation_colormap(self.num_classes)

            plt.figure(figsize=(16, 4))

            # Immagine di input
            plt.subplot(1, 3, 1)
            plt.imshow(image[0])
            plt.title("Input Image")
            plt.axis('off')

            # Ground truth
            plt.subplot(1, 3, 2)
            colored_label = apply_colormap(label.numpy(), colormap)
            plt.imshow(colored_label)
            plt.title("Ground Truth Mask")
            plt.axis('off')

            # Predizione
            plt.subplot(1, 3, 3)
            colored_pred = apply_colormap(y_pred[0], colormap)
            plt.imshow(colored_pred)
            plt.title("Predicted Mask")
            plt.axis('off')

            plt.tight_layout()
            plt.show()
            plt.close()



# Load data and augmentation

In [ ]:
training_data = np.load("/kaggle/input/datasetlomi/training_set_no_outliers.npz")
test_data = np.load("/kaggle/input/datasetlomi/test_set.npz")
images = training_data["images"]/255
labels = training_data["labels"]
print(images.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)

In [ ]:
def add_channel(image, label):
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.float32)
    image = tf.expand_dims(image, axis=-1)
    label = tf.expand_dims(label, axis=-1)
    return image, label

In [ ]:
"zoom"
@tf.function
def zoom(image, label, zoom_range=(0.8, 1.2)):
    # Genera un fattore di zoom casuale
    zoom_factor = tf.random.uniform([], zoom_range[0], zoom_range[1])

    # Ottieni le dimensioni originali
    original_height = tf.shape(image)[0]
    original_width = tf.shape(image)[1]

    # Calcola le nuove dimensioni dopo lo zoom
    new_height = tf.cast(tf.cast(original_height, tf.float32) * zoom_factor, tf.int32)
    new_width = tf.cast(tf.cast(original_width, tf.float32) * zoom_factor, tf.int32)

    # Ridimensiona l'immagine e la maschera
    zoomed_image = tf.image.resize(image, [new_height, new_width], method='bilinear')
    zoomed_label = tf.image.resize(label, [new_height, new_width], method='bilinear')  # Per maschere, meglio 'nearest'

    # Ritaglia o pad per riportare alle dimensioni originali
    cropped_image = tf.image.resize_with_crop_or_pad(zoomed_image, original_height, original_width)
    cropped_label = tf.image.resize_with_crop_or_pad(zoomed_label, original_height, original_width)

    return cropped_image, cropped_label

@tf.function
def random_zoom(image, label, thr):
    prob = tf.random.uniform([])

    image, label = tf.cond(
        prob < thr,
        lambda: (zoom(image, label)),
        lambda: (image, label)
    )
    return image, label

def zoom_lambda(dataset, thr):
    return dataset.map(
        lambda x, y: random_zoom(x, y, thr),
        num_parallel_calls=tf.data.AUTOTUNE
    )


In [ ]:
"contrast"
@tf.function
def random_contrast(image, label, thr):
    prob = tf.random.uniform([], seed=seed)
    factor = tf.random.uniform([], minval=-4, maxval=4, seed=seed)


    image, label = tf.cond(
        prob < thr,
        lambda: (tf.image.adjust_contrast(image, factor), label),
        lambda: (image, label)
    )
    return image, label

def contrast_lambda(dataset, thr):
    return dataset.map(
        lambda x, y: random_contrast(x, y, thr),
        num_parallel_calls=tf.data.AUTOTUNE
    )

In [ ]:
"Flip Left Right"
@tf.function
def random_flip_left_right(image, label, seed=None):
    if seed is None:
        seed = np.random.randint(0, 1000000)

    flip_prob = tf.random.uniform([], seed=seed)

    image, label = tf.cond(
        flip_prob > 0.5,
        lambda: (tf.image.flip_left_right(image), tf.image.flip_left_right(label)),
        lambda: (image, label)
    )

    return image, label

In [ ]:
"Flip Up Down"
@tf.function
def random_flip_up_down(image, label, seed=None):
    if seed is None:
        seed = np.random.randint(0, 1000000)

    flip_prob = tf.random.uniform([], seed=seed)

    image, label = tf.cond(
        flip_prob > 0.5,
        lambda: (tf.image.flip_up_down(image), tf.image.flip_up_down(label)),
        lambda: (image, label)
    )

    return image, label

In [ ]:
"Translation"
@tf.function
def translation(image, label, max_translation=0.2, seed=None):
    height = tf.shape(image)[0]
    width = tf.shape(image)[1]

    max_dx = tf.cast(max_translation * tf.cast(width, tf.float32), tf.int32)
    max_dy = tf.cast(max_translation * tf.cast(height, tf.float32), tf.int32)

    dx = tf.random.uniform([], -max_dx, max_dx + 1, dtype=tf.int32, seed=seed)
    dy = tf.random.uniform([], -max_dy, max_dy + 1, dtype=tf.int32, seed=seed)

    translated_image = tf.roll(image, shift=[dy, dx], axis=[0, 1])
    translated_label = tf.roll(label, shift=[dy, dx], axis=[0, 1])

    return translated_image, translated_label


@tf.function
def random_translation(image, label, max_translation=0.2, seed=None):
    if seed is None:
        seed = np.random.randint(0, 1000000)
    flip_prob = tf.random.uniform([], seed=seed)

    image, label = tf.cond(
        flip_prob > 0.7,
        lambda: (translation(image,label)),
        lambda: (image, label)
    )
    return image, label


In [ ]:
"Black Square"
@tf.function
def random_black_square(image, label, max_square_ratio=0.2, seed=seed):
    # Calcola altezza e larghezza dell'immagine
    height = tf.shape(image)[0]
    width = tf.shape(image)[1]

    # Determina la dimensione massima del quadrato, con un limite massimo di 32px
    max_square_size = tf.minimum(48, tf.cast(max_square_ratio * tf.minimum(tf.cast(height, tf.float32),
                                                                         tf.cast(width, tf.float32)), tf.int32))

    # Assicura che il quadrato abbia senso nelle dimensioni dell'immagine
    max_square_size = tf.maximum(15, tf.minimum(max_square_size, tf.minimum(height, width)))

    # Genera la dimensione del quadrato
    square_size = tf.random.uniform([], 1, max_square_size + 1, dtype=tf.int32, seed=seed)

    # Genera la posizione del quadrato
    max_x = tf.maximum(0, width - square_size)
    max_y = tf.maximum(0, height - square_size)

    start_x = tf.random.uniform([], 0, max_x + 1, dtype=tf.int32, seed=seed)
    start_y = tf.random.uniform([], 0, max_y + 1, dtype=tf.int32, seed=seed)

    # Crea una maschera per "cancellare" il quadrato
    mask = tf.ones_like(image)
    mask = tf.tensor_scatter_nd_update(
        mask,
        indices=tf.stack(tf.meshgrid(
            tf.range(start_y, start_y + square_size),
            tf.range(start_x, start_x + square_size)
        ), axis=-1),
        updates=tf.zeros([square_size, square_size, tf.shape(image)[-1]], dtype=image.dtype)
    )

    # Applica la maschera a immagine e label
    zeroed_image = image * mask
    zeroed_label = label * mask

    return zeroed_image, zeroed_label

@tf.function
def random_square(image, label, max_translation=0.2, seed=None):
    if seed is None:
        seed = np.random.randint(0, 1000000)
    prob = tf.random.uniform([], seed=seed)

    image, label = tf.cond(
        prob > 0.7,
        lambda: random_black_square(image,label),
        lambda: (image, label)
    )
    return image, label


In [ ]:
"random_negative"
@tf.function
def random_negative(image, label, seed=None):
    if seed is None:
        seed = np.random.randint(0, 1000000)
    flip_prob = tf.random.uniform([], seed=seed)

    image, label = tf.cond(
        flip_prob > 0.7,
        lambda: (1-image, label),
        lambda: (image, label)
    )

    return image, label

In [ ]:
def duplicate_dataset(images, labels, quantity = 1):
    indices = np.where(np.any(labels == 4, axis=(1, 2)))[0]

    images_to_duplicate = images[indices]
    labels_to_duplicate = labels[indices]

    images_to_duplicate = np.concatenate([images_to_duplicate, images_to_duplicate], axis=0)
    labels_to_duplicate = np.concatenate([labels_to_duplicate, labels_to_duplicate], axis=0)

    # Concatena ai dati originali
    images = np.concatenate([images, images_to_duplicate], axis=0)
    labels = np.concatenate([labels, labels_to_duplicate], axis=0)

    images = np.concatenate([images, images[:quantity]], axis=0)
    labels = np.concatenate([labels, labels[:quantity]], axis=0)

    return images, labels


In [ ]:
def to_datasett(X_train, y_train, augmentation = False, seed = seed, shuffle = True, batch_size = BATCH_SIZE, duplicate = False):
    if duplicate:
        X_train, y_train = duplicate_dataset(X_train, y_train, len(X_train))
        X_train, y_train = duplicate_dataset(X_train, y_train, len(X_train))
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=batch_size * 2, seed=seed)

    dataset = dataset.map(
                    lambda x, y: add_channel(x, y),
                    num_parallel_calls=tf.data.AUTOTUNE
                   )

    if augmentation:
        dataset = dataset.map(
                        lambda x, y: random_flip_up_down(x,y,seed),
                        num_parallel_calls=tf.data.AUTOTUNE
                    )
        dataset = dataset.map(
                        lambda x, y: random_flip_left_right(x,y,seed),
                        num_parallel_calls=tf.data.AUTOTUNE
                    )
        dataset = dataset.map(
                        lambda x, y: random_translation(x,y,seed),
                        num_parallel_calls=tf.data.AUTOTUNE
                    )
        dataset = dataset.map(
                        lambda x, y: random_negative(x,y,seed),
                        num_parallel_calls=tf.data.AUTOTUNE
                    )
        dataset = dataset.map(
                        lambda x, y: random_square(x,y,seed),
                        num_parallel_calls=tf.data.AUTOTUNE
                    )

        dataset = zoom_lambda(dataset,0.5)

        dataset = contrast_lambda(dataset,0.6)


    # Batch the data
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:
train_dataset = to_datasett(X_train, y_train, augmentation = True, duplicate = True)
val_dataset = to_datasett(X_val, y_val, augmentation = False, duplicate = True)

for image_batch, label_batch in train_dataset.take(1):  # Prendi un batch dal dataset
    print("Dimensioni del batch di immagini:", image_batch.shape)
    print("Dimensioni del batch di label:", label_batch.shape)

# **MODEL**

https://arxiv.org/pdf/2006.04868

In [ ]:
def encoder_block(input_tensor, filters, name="encoder_block"):
    """Blocchi encoder con convoluzioni, BatchNormalization, ReLU e riduzione delle dimensioni spaziali."""
    # Prima Convoluzione
    x = layers.Conv2D(filters, (3, 3), padding="same", name=f"{name}_conv1")(input_tensor)
    x = layers.BatchNormalization(name=f"{name}_bn1")(x)
    x = layers.ReLU(name=f"{name}_relu1")(x)

    # Seconda Convoluzione
    x = layers.Conv2D(filters, (3, 3), padding="same", name=f"{name}_conv2")(x)
    x = layers.BatchNormalization(name=f"{name}_bn2")(x)
    x = layers.ReLU(name=f"{name}_relu2")(x)

    # Riduzione della dimensione spaziale
    pooled = layers.MaxPooling2D((2, 2), strides=2, name=f"{name}_pool")(x)

    return pooled, pooled  # pooled: feature map dimezzata, x: feature map per le skip connections


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def aspp_block(input_tensor, filters, name="ASPP"):

    # Convoluzione 1x1
    conv1 = layers.Conv2D(filters, (1, 1), padding="same", activation="relu", name=f"{name}_conv1")(input_tensor)

    # Convoluzioni dilatate (3x3) con diversi tassi di dilatazione
    conv3_1 = layers.Conv2D(filters, (3, 3), dilation_rate=1, padding="same", activation="relu", name=f"{name}_conv3_6")(input_tensor)
    conv3_2 = layers.Conv2D(filters, (3, 3), dilation_rate=4, padding="same", activation="relu", name=f"{name}_conv3_12")(input_tensor)
    conv3_3 = layers.Conv2D(filters, (3, 3), dilation_rate=8, padding="same", activation="relu", name=f"{name}_conv3_18")(input_tensor)

    # Global Average Pooling
    global_avg = layers.GlobalAveragePooling2D(name=f"{name}_gap")(input_tensor)
    global_avg = layers.Reshape((1, 1, input_tensor.shape[-1]), name=f"{name}_reshape")(global_avg)
    global_avg = layers.Conv2D(filters, (1, 1), padding="same", activation="relu", name=f"{name}_conv_global")(global_avg)
    global_avg = layers.UpSampling2D(size=(input_tensor.shape[1], input_tensor.shape[2]), interpolation="bilinear", name=f"{name}_upsample")(global_avg)

    # Concatenazione
    x = layers.Concatenate(name=f"{name}_concat")([conv1, conv3_1, conv3_2, conv3_3, global_avg])

    # Convoluzione Finale per Ridurre i Canali
    x = layers.Conv2D(filters, (1, 1), padding="same", activation="relu", name=f"{name}_conv_out")(x)

    return x


In [ ]:
from tensorflow.keras.layers import Add, Concatenate, Dense, GlobalAveragePooling2D, Multiply, Lambda

def gated_skip_connection(x, skip, mode="add", gating="learnable", name="gated_connection"):
    if gating == "learnable":
        # Learnable scalars for gating
        alpha = tf.Variable(0.5, trainable=True, name=f"{name}_alpha")
        beta = tf.Variable(0.5, trainable=True, name=f"{name}_beta")

        # Normalize weights with a softmax
        alpha, beta = tf.nn.softmax([alpha, beta], axis=0)
        if mode == "add":
            output = Add(name=f"{name}_add")([alpha * x, beta * skip])
        elif mode == "concat":
            output = Concatenate(name=f"{name}_concat")([alpha * x, beta * skip])
        else:
            raise ValueError("Unsupported mode. Use 'add' or 'concat'.")

    elif gating == "dynamic":
        # Dynamic gating based on skip and decoder features
        combined = Concatenate(name=f"{name}_dynamic_concat")([x, skip])
        gate = GlobalAveragePooling2D(name=f"{name}_gap")(combined)
        gate = Dense(1, activation="sigmoid", name=f"{name}_gate")(gate)

        # Apply gating
        gate = Lambda(lambda z: tf.expand_dims(tf.expand_dims(z, axis=1), axis=1), name=f"{name}_expand")(gate)
        gated_skip = Multiply(name=f"{name}_multiply")([skip, gate])
        output = Add(name=f"{name}_add_dynamic")([x, gated_skip])

    else:
        raise ValueError("Unsupported gating mode. Use 'learnable' or 'dynamic'.")

    return output


In [ ]:
from tensorflow.keras import layers, Model

def network1(input_shape=(64, 128, 1), num_classes=5, filter_factor=0.5):
    inputs = layers.Input(shape=input_shape)

    # Blocco 1
    x = layers.Conv2D(int(64 * filter_factor), (3, 3), padding="same", activation="relu", name="block1_conv1")(inputs)
    x = layers.Conv2D(int(64 * filter_factor), (3, 3), padding="same", activation="relu", name="block1_conv2")(x)
    block1 = x
    x = layers.MaxPooling2D((2, 2), strides=2, name="block1_pool")(x)

    # Blocco 2
    x = layers.Conv2D(int(128 * filter_factor), (3, 3), padding="same", activation="relu", name="block2_conv1")(x)
    x = layers.Conv2D(int(128 * filter_factor), (3, 3), padding="same", activation="relu", name="block2_conv2")(x)
    block2 = x
    x = layers.MaxPooling2D((2, 2), strides=2, name="block2_pool")(x)

    # Blocco 3
    x = layers.Conv2D(int(256 * filter_factor), (3, 3), padding="same", activation="relu", name="block3_conv1")(x)
    x = layers.Dropout(0.3, name="block3_dropout1")(x)
    x = layers.Conv2D(int(256 * filter_factor), (3, 3), padding="same", activation="relu", name="block3_conv2")(x)
    x = layers.Dropout(0.3, name="block3_dropout2")(x)
    x = layers.Conv2D(int(256 * filter_factor), (3, 3), padding="same", activation="relu", name="block3_conv3")(x)
    block3 = x
    x = layers.MaxPooling2D((2, 2), strides=2, name="block3_pool")(x)

    # Blocco 4
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="block4_conv1")(x)
    x = layers.Dropout(0.3, name="block4_dropout1")(x)
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="block4_conv2")(x)
    x = layers.Dropout(0.3, name="block4_dropout2")(x)
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="block4_conv3")(x)
    block4 = x
    x = layers.MaxPooling2D((2, 2), strides=2, name="block4_pool")(x)

    # Blocco 5
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="block5_conv1")(x)
    x = layers.Dropout(0.3, name="block5_dropout1")(x)
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="block5_conv2")(x)
    x = layers.Dropout(0.3, name="block5_dropout2")(x)
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="block5_conv3")(x)
    block5 = x
    x = layers.MaxPooling2D((2, 2), strides=2, name="block5_pool")(x)

    # Bottleneck - Secondo Metodo
    dilation_1 = layers.Conv2D(int(256 * filter_factor), (3, 3), dilation_rate=1, padding="same", activation="relu", name="bottleneck_dilation1")(x)
    dilation_2 = layers.Conv2D(int(256 * filter_factor), (3, 3), dilation_rate=2, padding="same", activation="relu", name="bottleneck_dilation2")(x)
    dilation_3 = layers.Conv2D(int(256 * filter_factor), (3, 3), dilation_rate=4, padding="same", activation="relu", name="bottleneck_dilation3")(x)
    global_pool = layers.GlobalAveragePooling2D(name="bottleneck_global_pool")(x)
    global_pool = layers.Reshape((1, 1, int(512 * filter_factor)), name="bottleneck_global_reshape")(global_pool)
    global_pool = layers.Conv2D(int(256 * filter_factor), (1, 1), activation="relu", name="bottleneck_global_conv")(global_pool)
    global_pool = layers.UpSampling2D(size=(x.shape[1], x.shape[2]), interpolation="bilinear", name="bottleneck_global_upsample")(global_pool)

    # Concatenazione Bottleneck
    x = layers.Concatenate(name="bottleneck_concat")([dilation_1, dilation_2, dilation_3, global_pool])
    x = layers.Conv2D(int(512 * filter_factor), (1, 1), padding="same", activation="relu", name="bottleneck_output")(x)

    # Decoder (Upsampling)
    x = layers.UpSampling2D((2, 2), name="decoder_network1_upsample0")(x)
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="decoder_network1_conv1")(x)

    # Gated Skip Connection for Block 5
    x = gated_skip_connection(x, block5, mode="add", gating="learnable", name="decoder_network1_gated1")
    x = layers.Conv2D(int(512 * filter_factor), (3, 3), padding="same", activation="relu", name="decoder_network1_conv2")(x)
    x = layers.BatchNormalization(name="decoder_network1_bn1")(x)

    # Gated Skip Connection for Block 4
    x = layers.UpSampling2D((2, 2), name="decoder_network1_upsample1")(x)
    x = gated_skip_connection(x, block4, mode="add", gating="learnable", name="decoder_network1_gated2")
    x = layers.Conv2D(int(256 * filter_factor), (3, 3), padding="same", activation="relu", name="decoder_network1_conv3")(x)
    x = layers.BatchNormalization(name="decoder_network1_bn2")(x)

    # Gated Skip Connection for Block 3
    x = layers.UpSampling2D((2, 2), name="decoder_network1_upsample2")(x)
    x = gated_skip_connection(x, block3, mode="add", gating="learnable", name="decoder_network1_gated3")
    x = layers.Conv2D(int(128 * filter_factor), (3, 3), padding="same", activation="relu", name="decoder_network1_conv4")(x)
    x = layers.BatchNormalization(name="decoder_network1_bn3")(x)

    # Gated Skip Connection for Block 2
    x = layers.UpSampling2D((2, 2), name="decoder_network1_upsample3")(x)
    x = gated_skip_connection(x, block2, mode="add", gating="learnable", name="decoder_network1_gated4")
    x = layers.Conv2D(int(64 * filter_factor), (3, 3), padding="same", activation="relu", name="decoder_network1_conv5")(x)
    x = layers.BatchNormalization(name="decoder_network1_bn4")(x)

    # Final Upsampling
    x = layers.UpSampling2D((2, 2), name="decoder_network1_upsample4")(x)
    outputs = layers.Conv2D(num_classes, (1, 1), activation="softmax", name="output1")(x)

    model = tfk.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
model = network1()

# Print a detailed summary of the model with expanded nested layers and trainable parameters.
#model.summary(expand_nested=True, show_trainable=True)

# Generate and display a graphical representation of the model architecture.
#tf.keras.utils.plot_model(model, show_trainable=True, expand_nested=True, dpi=70)

#tf.keras.utils.plot_model(model, show_trainable=True, expand_nested=True, dpi=70)

# Mean Intersection over union

In [ ]:
class MeanIntersectionOverUnion(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes, labels_to_exclude=[0], name="mean_iou", dtype=None, **kwargs):
        """
        Aggiunto **kwargs per gestire parametri inattesi come `ignore_class`.
        """
        super(MeanIntersectionOverUnion, self).__init__(num_classes=num_classes, name=name, dtype=dtype)
        if labels_to_exclude is None:
            labels_to_exclude = [0]
        self.labels_to_exclude = labels_to_exclude

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.math.argmax(y_pred, axis=-1)
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])

        for label in self.labels_to_exclude:
            mask = tf.not_equal(y_true, label)
            y_true = tf.boolean_mask(y_true, mask)
            y_pred = tf.boolean_mask(y_pred, mask)

        return super().update_state(y_true, y_pred, sample_weight)

# Registra la classe personalizzata
tf.keras.utils.get_custom_objects()["MeanIntersectionOverUnion"] = MeanIntersectionOverUnion


# Losses

In [ ]:
labels_flat = y_train.reshape(-1)  # Appiattisci l'array per considerare tutti i pixel

# Conta la frequenza di ogni classe
num_classes = 5  # Modifica in base al numero di classi nel tuo dataset
labels_flat = labels_flat.astype(int)  # Converte i dati in interi

# Conta la frequenza di ogni classe
class_frequencies = np.bincount(labels_flat, minlength=num_classes)

# Stampa le frequenze
print(f"Frequenze delle classi: {class_frequencies}")

total_samples = np.sum(class_frequencies)

# Calcola i pesi di classe
class_weights = total_samples / (len(class_frequencies) * class_frequencies)


# Conversione in dizionario per TensorFlow
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print(f"Dizionario dei pesi di classe: {class_weights_dict}")

# Funzione di perdita con pesi di classe
loss = tf.keras.losses.SparseCategoricalCrossentropy()



def weighted_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int32)  # Assicurati che i valori siano interi
    weights = tf.gather(class_weights, y_true)  # Recupera i pesi in base alle etichette
    weights = tf.cast(weights, tf.float32)  # Converte i pesi in float32
    unweighted_loss = loss(y_true, y_pred)
    weighted_loss = unweighted_loss * weights
    return tf.reduce_mean(weighted_loss)

In [ ]:
class ComboLoss:
    def __init__(self, alpha=0.5, dice_smooth=1e-7, class_weights=None):
        self.alpha = alpha
        self.dice_smooth = dice_smooth
        self.class_weights = class_weights

    def dice_loss(self, y_true, y_pred):
        y_true = tf.cast(tf.one_hot(tf.cast(y_true[..., 0], tf.int32), depth=y_pred.shape[-1]), tf.float32)
        intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
        cardinality = tf.reduce_sum(y_true, axis=[1, 2]) + tf.reduce_sum(y_pred, axis=[1, 2])
        dice = (2. * intersection + self.dice_smooth) / (cardinality + self.dice_smooth)
        return 1 - tf.reduce_mean(dice)

    def weighted_loss(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)  # Assicurati che i valori siano interi
        weights = tf.gather(class_weights, y_true)  # Recupera i pesi in base alle etichette
        weights = tf.cast(weights, tf.float32)  # Converte i pesi in float32
        unweighted_loss = loss(y_true, y_pred)
        weighted_loss = unweighted_loss * weights
        return tf.reduce_mean(weighted_loss)

    def __call__(self, y_true, y_pred):
        wce = self.weighted_loss(y_true, y_pred)
        dice = self.dice_loss(y_true, y_pred)
        return self.alpha * wce + (1 - self.alpha) * dice

    def get_config(self):
        return {
            "alpha": self.alpha,
            "dice_smooth": self.dice_smooth,
            "class_weights": self.class_weights
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)


combo_loss = ComboLoss(alpha=0.5, class_weights=class_weights)

In [ ]:
from tensorflow.keras import backend as K

def iou_loss(y_true, y_pred, smooth=1e-7):
    """
    Loss basata su Intersection over Union (IoU).

    Args:
        y_true (Tensor): Ground truth, dimensione (batch, height, width, num_classes).
        y_pred (Tensor): Predizioni del modello, stessa dimensione di y_true.
        smooth (float): Fattore di stabilizzazione per evitare divisioni per zero.

    Returns:
        Tensor: Valore medio della IoU Loss.
    """
    if y_true.shape[-1] != y_pred.shape[-1]:
        y_true = tf.one_hot(tf.cast(y_true[..., 0], tf.int32), depth=tf.shape(y_pred)[-1])

    # Calcolo di Intersection e Union
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2]) - intersection
    iou = (intersection + smooth) / (union + smooth)

    # Loss come complemento dell'IoU
    return 1 - tf.reduce_mean(iou)


# Compilation and training

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),
    loss=combo_loss,
    metrics=[
        "accuracy",
        MeanIntersectionOverUnion(num_classes=5, labels_to_exclude=[0])
    ]
)

In [ ]:
patience = 50
epochs = 1000

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=patience,
    restore_best_weights=True
)

viz_callback = VizCallback(val_dataset, frequency=5, num_classes=5)

In [ ]:
history = model.fit(
    train_dataset,   # Dataset con immagini e label
    validation_data=val_dataset,
    epochs=epochs,
    verbose=1,
    callbacks=[early_stopping, viz_callback]
)

# Calculate and print the final validation accuracy
#final_val_meanIoU = round(max(history['val_mean_iou'])* 100, 2)
#print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}%')

# Definisci il percorso di salvataggio nella directory di lavoro
model.save("/kaggle/working/FirstPart.keras")

# Delete the model to free up resources
#del model

# Testing

In [ ]:
model_filename = "FirstPart.keras"

X_test = np.load("/kaggle/input/datasetlomi/test_set.npz")
model = tf.keras.models.load_model(
    "/kaggle/working/FirstPart.keras",
    custom_objects={'ComboLoss': ComboLoss}
)

preds = model.predict(X_test["images"]/255)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

In [ ]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [ ]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply

def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1  # Canali sull'ultima dimensione
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    # Squeeze: Global Average Pooling
    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)

    # Excitation: Bottleneck + Sigmoid scaling
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    # Scaling i canali dell'input
    x = Multiply()([init, se])
    return x


In [ ]:
from tensorflow.keras.regularizers import l2

def network2(input_shape=(64, 128, 1), num_classes=5, dropout_rate=0.3):
    # Carica il modello pre-addestrato
    pretrained_model = tf.keras.models.load_model(
        "/kaggle/working/FirstPart.keras",
        custom_objects={'weighted_loss': weighted_loss}
    )

    # Blocca l'addestramento del modello pre-addestrato
    for layer in pretrained_model.layers:
        layer.trainable = False

    # Recupera gli skip connection e l'output
    block1 = pretrained_model.get_layer("block1_conv2").output
    block2 = pretrained_model.get_layer("block2_conv2").output
    block3 = pretrained_model.get_layer("block3_conv3").output
    block4 = pretrained_model.get_layer("block4_conv3").output
    block5 = pretrained_model.get_layer("block5_conv3").output
    outputs1 = pretrained_model.get_layer("output1").output

    print("Input del modello salvato:", pretrained_model.input_shape)
    print("Output del modello salvato:", outputs1.shape)

    # FUSIONE
    x = layers.Conv2D(1, (1, 1), activation="softmax", name="output1_compressed")(outputs1)
    multiplication = layers.Multiply(name="multiplication")([x, pretrained_model.input])
    print("Dopo moltiplicazione: ", multiplication.shape)

    # SECONDO ENCODER
    x, skip1 = encoder_block(multiplication, 64, name="encoder_block1_network2")
    x = squeeze_excite_block(x)
    x = layers.Dropout(dropout_rate, name="dropout_encoder1")(x)  # Dropout aggiunto
    print("x1: ", x.shape)

    x, skip2 = encoder_block(x, 128, name="encoder_block2_network2")
    x = squeeze_excite_block(x)
    x = layers.Dropout(dropout_rate, name="dropout_encoder2")(x)
    print("x2: ", x.shape)

    x, skip3 = encoder_block(x, 256, name="encoder_block3_network2")
    x = squeeze_excite_block(x)
    x = layers.SpatialDropout2D(dropout_rate, name="spatial_dropout_encoder3")(x)
    print("x3: ", x.shape)

    output_encoder2, skip4 = encoder_block(x, 512, name="encoder_block4_network2")
    x = squeeze_excite_block(x)
    x = layers.SpatialDropout2D(dropout_rate, name="spatial_dropout_encoder4")(output_encoder2)
    print("x4: ", x.shape)

    passo_attraverso_ASPP = aspp_block(x, 64, "PIPPO2")

    # SECONDO DECODER
    x = layers.Conv2D(512, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(1e-4))(passo_attraverso_ASPP)
    x = layers.Concatenate()([passo_attraverso_ASPP, block5, output_encoder2])
    x = layers.Conv2D(512, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(1e-4))(x)
    x = layers.Dropout(dropout_rate, name="dropout_decoder1")(x)
    print("Input secondo decoder x1: ", x.shape)

    # Block 4 -> Block 3
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, block4, skip3])
    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(1e-4))(x)
    x = layers.Dropout(dropout_rate, name="dropout_decoder2")(x)
    print("Input secondo decoder x2: ", x.shape)

    # Block 3 -> Block 2
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, block3, skip2])
    x = layers.Conv2D(128, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate, name="dropout_decoder3")(x)
    print("Input secondo decoder x3: ", x.shape)

    # Block 2 -> Block 1
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, block2, skip1])
    x = layers.Conv2D(64, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(1e-4))(x)
    x = layers.Dropout(dropout_rate, name="dropout_decoder4")(x)
    print("Input secondo decoder x4: ", x.shape)

    # Output finale
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(num_classes, (1, 1), activation="softmax", name="output2")(x)

    # Output combinato
    final_output = layers.Concatenate()([outputs1, x])
    final_output = layers.Conv2D(5, (1, 1), activation="softmax", name="final_output")(final_output)

    model = tfk.Model(inputs=pretrained_model.input, outputs=final_output)
    print("Final output: ", final_output.shape)

    return model


In [ ]:
model = network2()
#model.summary(expand_nested=True, show_trainable=True)
#tf.keras.utils.plot_model(model, show_trainable=True, expand_nested=True, dpi=70)

In [ ]:
model.compile(
        loss=iou_loss,
        optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),
        metrics=[
            "accuracy",
            MeanIntersectionOverUnion(num_classes=5, labels_to_exclude=[0])
        ]
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_iou',
    mode='max',
    patience=50,
    restore_best_weights=True
)

viz_callback = VizCallback(val_dataset, frequency=5, num_classes=5)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=[early_stopping, viz_callback],
    verbose=1
).history

# Calculate and print the final validation accuracy
#final_val_meanIoU = round(max(history['val_mean_iou'])* 100, 2)
#print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}%')

# Definisci il percorso di salvataggio nella directory di lavoro
model.save("/kaggle/working/SecondPart.keras")

# Delete the model to free up resources
del model

In [ ]:
model_filename = "SecondPart.keras"

X_test = np.load("/kaggle/input/datasetlomi/test_set.npz")
model = tf.keras.models.load_model(
    "/kaggle/working/SecondPart.keras",
    custom_objects={'iou_loss': iou_loss}
)

preds = model.predict(X_test["images"]/255)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

In [ ]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [ ]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)